In [1]:
from os.path import splitext, exists, join
from tempfile import TemporaryDirectory

In [3]:
from spleeter.audio.adapter import get_default_audio_adapter
from spleeter.separator import Separator

In [4]:
TEST_AUDIO_DESCRIPTOR = 'audio_example.mp3'
TEST_AUDIO_BASENAME = splitext(TEST_AUDIO_DESCRIPTOR)

In [5]:
TEST_AUDIO_BASENAME

('audio_example', '.mp3')

In [6]:
adapter = get_default_audio_adapter()
waveform, _ = adapter.load(TEST_AUDIO_DESCRIPTOR)

In [7]:
separator = Separator('spleeter:2stems')

In [8]:
predictor = separator._get_predictor()

INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models\\2stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023682E84888>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

INFO:tensorflow:Apply unet for vocals_spectrogram
INFO:tensorflow:Apply unet for accompaniment_spectrogram
INFO:tensorflow:Done calling

In [9]:
predictor

SavedModelPredictor with feed tensors {'waveform': <tf.Tensor 'Placeholder:0' shape=(?, 2) dtype=float32>, 'audio_id': <tf.Tensor 'Placeholder_1:0' shape=<unknown> dtype=string>, 'mix_stft': <tf.Tensor 'transpose_1:0' shape=(?, 2049, 2) dtype=complex64>, 'mix_spectrogram': <tf.Tensor 'strided_slice_3:0' shape=(?, 512, 1024, 2) dtype=float32>} and fetch_tensors {'vocals': <tf.Tensor 'strided_slice_13:0' shape=(?, 2) dtype=float32>, 'accompaniment': <tf.Tensor 'strided_slice_23:0' shape=(?, 2) dtype=float32>, 'audio_id': <tf.Tensor 'Placeholder_1:0' shape=<unknown> dtype=string>}

In [12]:
prediction = predictor({
            'waveform': waveform,
            'audio_id': ''})
prediction.pop('audio_id')

array('', dtype=object)

In [11]:
prediction

{'vocals': array([[ 0.0000000e+00,  0.0000000e+00],
        [-1.7786974e-13, -4.5078372e-13],
        [ 2.1400206e-12, -3.6755854e-12],
        ...,
        [ 3.8986618e-06,  3.6624367e-06],
        [ 4.5178203e-06,  2.6510040e-06],
        [ 4.4329231e-06,  1.9839292e-06]], dtype=float32),
 'accompaniment': array([[ 0.0000000e+00,  0.0000000e+00],
        [-4.3002640e-13,  1.3992512e-12],
        [-4.0228563e-12,  8.1480378e-12],
        ...,
        [-1.4976256e-05, -2.7992608e-05],
        [-1.9667097e-05, -1.8059734e-05],
        [-2.1309363e-05, -1.9806143e-06]], dtype=float32),
 'audio_id': array('', dtype=object)}

In [13]:
separator.separate_to_file(TEST_AUDIO_DESCRIPTOR, "output")

In [14]:
adapter.save('output/vocals.wav', prediction['vocals'], _)

INFO:spleeter:File output/vocals.wav written


In [15]:
adapter.save('output/accompaniment.wav', prediction['accompaniment'], _)

INFO:spleeter:File output/accompaniment.wav written
